# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'GQGE(A)NTKM/T C EBALE LRASANDLSOS',
 'calculationPrice': 'close',
 'open': 139,
 'openTime': 1626309883929,
 'openSource': 'lficaofi',
 'close': 135.13,
 'closeTime': 1646531749252,
 'closeSource': 'oifcailf',
 'high': 136.44,
 'highTime': 1632504693544,
 'highSource': 'ey5t iad1 mp unecdereil',
 'low': 136.55,
 'lowTime': 1686851226191,
 'lowSource': 'inramledtu5cpi1y   deee',
 'latestPrice': 132.97,
 'latestSource': 'Close',
 'latestTime': 'December 31, 2020',
 'latestUpdate': 1619425866038,
 'latestVolume': 101571430,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 138.92,
 'delayedPriceTime': 1611200227261,
 'oddLotDelayedPrice': 135.37,
 'oddLotDelayedPriceTime': 1685981823648,
 'extendedPrice': 135.34,
 'extendedChange': 0.05,
 'extendedChangePercent': 0.00039,
 'extendedPriceTime': 1668243324886,
 'previousClose': 138.09,
 'previousVolume': 97736615,
 'change': -1.

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['latestPrice']
pe_ratio = data["peRatio"]
print(price, pe_ratio)

132.97 40.59


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [15]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['peRatio'], 'N/A'], index=my_columns), ignore_index=True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,120.35,54.63,N/A
1,AAL,16.17,-1.15,N/A
2,AAP,158.43,23.96,N/A
3,AAPL,136.70,41.23,N/A
4,ABBV,107.50,23.8,N/A
...,...,...,...,...
500,YUM,113.64,32.7,N/A
501,ZBH,154.70,-210.8,N/A
502,ZBRA,397.49,45.17,N/A
503,ZION,45.10,18.74,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [16]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True, ascending=False)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [17]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,12.33,14886.7,N/A
1,FIS,145.27,2904,N/A
2,GLW,38.00,304.13,N/A
3,KMI,13.68,298.64,N/A
4,MAR,136.87,248.93,N/A
5,IQV,181.99,199.64,N/A
6,REG,45.85,171.91,N/A
7,CMG,1415.96,171.75,N/A
8,EL,266.29,167.3,N/A
9,GPN,216.76,161.18,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [18]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [19]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [22]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,12.33,14886.7,1622
1,FIS,145.27,2904,137
2,GLW,38.00,304.13,526
3,KMI,13.68,298.64,1461
4,MAR,136.87,248.93,146
5,IQV,181.99,199.64,109
6,REG,45.85,171.91,436
7,CMG,1415.96,171.75,14
8,EL,266.29,167.3,75
9,GPN,216.76,161.18,92


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [26]:
symbol = "AAPL"
batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_val = data[symbol]['advanced-stats']['enterpriseValue']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda_ratio = enterprise_val / data[symbol]['advanced-stats']['EBITDA']

# Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gp = enterprise_val / data[symbol]['advanced-stats']['grossProfit']

print(pe_ratio, pb_ratio, ps_ratio, ev_to_ebitda_ratio, ev_to_gp)

42.02 34.56 8.24 28.9195477229065 21.23095776954761


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [27]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )


In [28]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.97,N/A,54.18,N/A,7.6,N/A,7.16,N/A,33.089033,N/A,13.026655,N/A,N/A
1,AAL,15.86,N/A,-1.17,N/A,-1.74,N/A,0.388,N/A,-6.184268,N/A,1.431727,N/A,N/A
2,AAP,160.62,N/A,23.54,N/A,2.96,N/A,1.13,N/A,11.287689,N/A,2.471893,N/A,N/A
3,AAPL,138.65,N/A,42.1,N/A,36.08,N/A,8.51,N/A,29.289193,N/A,21.018637,N/A,N/A
4,ABBV,108.83,N/A,24.4,N/A,12.59,N/A,4.79,N/A,16.400071,N/A,9.726260,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.92,N/A,33.9,N/A,-4.13,N/A,5.99,N/A,24.434218,N/A,10.831442,N/A,N/A
501,ZBH,158.70,N/A,-209.67,N/A,2.82,N/A,4.71,N/A,26.235121,N/A,7.718650,N/A,N/A
502,ZBRA,391.24,N/A,44.59,N/A,10.67,N/A,4.76,N/A,30.892685,N/A,11.478661,N/A,N/A
503,ZION,44.24,N/A,18.51,N/A,1,N/A,2.45,N/A,10.943340,N/A,2.768656,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [30]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,213.630,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,234.570,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.260,N/A,14.3,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.160,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.730,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,151.030,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1816.460,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [31]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [32]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [33]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.859406
1      0.140594
2      0.451485
3      0.754455
4      0.463366
         ...   
500    0.665347
501    0.019802
502    0.774257
503    0.362376
504    0.825743
Name: PE Percentile, Length: 505, dtype: object
0       0.730693
1      0.0455446
2       0.415842
3       0.964356
4       0.849505
         ...    
500    0.0415842
501     0.388119
502     0.821782
503    0.0980198
504     0.946535
Name: PB Percentile, Length: 505, dtype: object
0        0.80198
1      0.0277228
2       0.165347
3       0.849505
4       0.685149
         ...    
500     0.752475
501     0.675248
502     0.681188
503      0.40297
504     0.934653
Name: PS Percentile, Length: 505, dtype: object
0       0.843564
1      0.0277228
2       0.281188
3       0.809901
4        0.50297
         ...    
500     0.744554
501     0.772277
502     0.829703
503     0.255446
504     0.855446
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.794059
1      0.0613861
2       0.124752
3       0.938

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.97,N/A,54.18,0.859406,7.60,0.730693,7.160,0.80198,33.089033,0.843564,13.026655,0.794059,N/A
1,AAL,15.86,N/A,-1.17,0.140594,-1.74,0.0455446,0.388,0.0277228,-6.184268,0.0277228,1.431727,0.0613861,N/A
2,AAP,160.62,N/A,23.54,0.451485,2.96,0.415842,1.130,0.165347,11.287689,0.281188,2.471893,0.124752,N/A
3,AAPL,138.65,N/A,42.10,0.754455,36.08,0.964356,8.510,0.849505,29.289193,0.809901,21.018637,0.938614,N/A
4,ABBV,108.83,N/A,24.40,0.463366,12.59,0.849505,4.790,0.685149,16.400071,0.50297,9.726260,0.655446,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.92,N/A,33.90,0.665347,-4.13,0.0415842,5.990,0.752475,24.434218,0.744554,10.831442,0.70099,N/A
501,ZBH,158.70,N/A,-209.67,0.019802,2.82,0.388119,4.710,0.675248,26.235121,0.772277,7.718650,0.489109,N/A
502,ZBRA,391.24,N/A,44.59,0.774257,10.67,0.821782,4.760,0.681188,30.892685,0.829703,11.478661,0.720792,N/A
503,ZION,44.24,N/A,18.51,0.362376,1.00,0.0980198,2.450,0.40297,10.943340,0.255446,2.768656,0.144554,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [34]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.97,N/A,54.18,0.859406,7.60,0.730693,7.160,0.80198,33.089033,0.843564,13.026655,0.794059,0.805941
1,AAL,15.86,N/A,-1.17,0.140594,-1.74,0.0455446,0.388,0.0277228,-6.184268,0.0277228,1.431727,0.0613861,0.0605941
2,AAP,160.62,N/A,23.54,0.451485,2.96,0.415842,1.130,0.165347,11.287689,0.281188,2.471893,0.124752,0.287723
3,AAPL,138.65,N/A,42.10,0.754455,36.08,0.964356,8.510,0.849505,29.289193,0.809901,21.018637,0.938614,0.863366
4,ABBV,108.83,N/A,24.40,0.463366,12.59,0.849505,4.790,0.685149,16.400071,0.50297,9.726260,0.655446,0.631287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.92,N/A,33.90,0.665347,-4.13,0.0415842,5.990,0.752475,24.434218,0.744554,10.831442,0.70099,0.58099
501,ZBH,158.70,N/A,-209.67,0.019802,2.82,0.388119,4.710,0.675248,26.235121,0.772277,7.718650,0.489109,0.468911
502,ZBRA,391.24,N/A,44.59,0.774257,10.67,0.821782,4.760,0.681188,30.892685,0.829703,11.478661,0.720792,0.765545
503,ZION,44.24,N/A,18.51,0.362376,1.00,0.0980198,2.450,0.40297,10.943340,0.255446,2.768656,0.144554,0.252673


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [35]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [36]:
portfolio_input()

In [37]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.47,852,5.12,0.156436,0.4364,0.049505,0.3918,0.029703,2.610494,0.0336634,0.386009,0.0039604,0.0546535
1,AIG,39.74,503,-6.94,0.10495,0.5159,0.0574257,0.7469,0.0653465,4.278409,0.0376238,0.712462,0.0138614,0.0558416
2,AAL,15.86,1261,-1.17,0.140594,-1.7400,0.0455446,0.3880,0.0277228,-6.184268,0.0277228,1.431727,0.0613861,0.0605941
3,HFC,26.48,755,-27.20,0.0673267,0.8124,0.0792079,0.3015,0.0138614,6.225501,0.0653465,2.153933,0.110891,0.0673267
4,MET,47.27,423,8.10,0.168317,0.6037,0.0613861,0.6460,0.0554455,4.458363,0.0415842,0.651160,0.0118812,0.0677228
5,F,9.16,2183,-17.08,0.0811881,1.1400,0.126733,0.2694,0.0118812,4.648893,0.0435644,2.074420,0.10297,0.0732673
6,HIG,50.14,398,10.37,0.194059,1.0000,0.0980198,0.8585,0.0910891,4.351823,0.039604,0.846171,0.0217822,0.0889109
7,ALL,110.96,180,8.00,0.166337,1.3400,0.166337,0.7614,0.0693069,2.516215,0.0316832,0.725711,0.0158416,0.089901
8,BA,224.13,89,-27.37,0.0653465,-10.3700,0.0316832,2.0800,0.353465,-29.125299,0.0138614,-205.818396,0.0019802,0.0932673
9,KSS,41.43,482,-61.20,0.0475248,1.3500,0.171287,0.3860,0.0257426,9.684562,0.194059,1.431336,0.0594059,0.099604


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [38]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [39]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [40]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [41]:
writer.save()